# 使用生成对抗网络生成图片

读入CIFAR-10数据

In [1]:
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torchvision.utils import save_image

dataset = CIFAR10(root='./data', download=True,
        transform=transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

for batch_idx, data in enumerate(dataloader):
    real_images, _ = data
    batch_size = real_images.size(0)
    print ('#{} has {} images.'.format(batch_idx, batch_size))
    if batch_idx % 100 == 0:
        path = './data/CIFAR10_shuffled_batch{:03d}.png'.format(batch_idx)
        save_image(real_images, path, normalize=True)

Files already downloaded and verified
#0 has 64 images.
#1 has 64 images.
#2 has 64 images.
#3 has 64 images.
#4 has 64 images.
#5 has 64 images.
#6 has 64 images.
#7 has 64 images.
#8 has 64 images.
#9 has 64 images.
#10 has 64 images.
#11 has 64 images.
#12 has 64 images.
#13 has 64 images.
#14 has 64 images.
#15 has 64 images.
#16 has 64 images.
#17 has 64 images.
#18 has 64 images.
#19 has 64 images.
#20 has 64 images.
#21 has 64 images.
#22 has 64 images.
#23 has 64 images.
#24 has 64 images.
#25 has 64 images.
#26 has 64 images.
#27 has 64 images.
#28 has 64 images.
#29 has 64 images.
#30 has 64 images.
#31 has 64 images.
#32 has 64 images.
#33 has 64 images.
#34 has 64 images.
#35 has 64 images.
#36 has 64 images.
#37 has 64 images.
#38 has 64 images.
#39 has 64 images.
#40 has 64 images.
#41 has 64 images.
#42 has 64 images.
#43 has 64 images.
#44 has 64 images.
#45 has 64 images.
#46 has 64 images.
#47 has 64 images.
#48 has 64 images.
#49 has 64 images.
#50 has 64 images.
#51

#418 has 64 images.
#419 has 64 images.
#420 has 64 images.
#421 has 64 images.
#422 has 64 images.
#423 has 64 images.
#424 has 64 images.
#425 has 64 images.
#426 has 64 images.
#427 has 64 images.
#428 has 64 images.
#429 has 64 images.
#430 has 64 images.
#431 has 64 images.
#432 has 64 images.
#433 has 64 images.
#434 has 64 images.
#435 has 64 images.
#436 has 64 images.
#437 has 64 images.
#438 has 64 images.
#439 has 64 images.
#440 has 64 images.
#441 has 64 images.
#442 has 64 images.
#443 has 64 images.
#444 has 64 images.
#445 has 64 images.
#446 has 64 images.
#447 has 64 images.
#448 has 64 images.
#449 has 64 images.
#450 has 64 images.
#451 has 64 images.
#452 has 64 images.
#453 has 64 images.
#454 has 64 images.
#455 has 64 images.
#456 has 64 images.
#457 has 64 images.
#458 has 64 images.
#459 has 64 images.
#460 has 64 images.
#461 has 64 images.
#462 has 64 images.
#463 has 64 images.
#464 has 64 images.
#465 has 64 images.
#466 has 64 images.
#467 has 64 images.


搭建生成网络和鉴别网络

In [2]:
import torch.nn as nn

# 搭建生成网络
latent_size = 64 # 潜在大小
n_channel = 3 # 输出通道数
n_g_feature = 64 # 生成网络隐藏层大小
gnet = nn.Sequential( 
        # 输入大小 = (64, 1, 1)
        nn.ConvTranspose2d(latent_size, 4 * n_g_feature, kernel_size=4,
                bias=False),
        nn.BatchNorm2d(4 * n_g_feature),
        nn.ReLU(),
        # 大小 = (256, 4, 4)
        nn.ConvTranspose2d(4 * n_g_feature, 2 * n_g_feature, kernel_size=4,
                stride=2, padding=1, bias=False),
        nn.BatchNorm2d(2 * n_g_feature),
        nn.ReLU(),
        # 大小 = (128, 8, 8)
        nn.ConvTranspose2d(2 * n_g_feature, n_g_feature, kernel_size=4,
                stride=2, padding=1, bias=False),
        nn.BatchNorm2d(n_g_feature),
        nn.ReLU(),
        # 大小 = (64, 16, 16)
        nn.ConvTranspose2d(n_g_feature, n_channel, kernel_size=4,
                stride=2, padding=1),
        nn.Sigmoid(),
        # 图片大小 = (3, 32, 32)
        )
print (gnet)

# 搭建鉴别网络
n_d_feature = 64 # 鉴别网络隐藏层大小
dnet = nn.Sequential( 
        # 图片大小 = (3, 32, 32)
        nn.Conv2d(n_channel, n_d_feature, kernel_size=4,
                stride=2, padding=1),
        nn.LeakyReLU(0.2),
        # 大小 = (64, 16, 16)
        nn.Conv2d(n_d_feature, 2 * n_d_feature, kernel_size=4,
                stride=2, padding=1, bias=False),
        nn.BatchNorm2d(2 * n_d_feature),
        nn.LeakyReLU(0.2),
        # 大小 = (128, 8, 8)
        nn.Conv2d(2 * n_d_feature, 4 * n_d_feature, kernel_size=4,
                stride=2, padding=1, bias=False),
        nn.BatchNorm2d(4 * n_d_feature),
        nn.LeakyReLU(0.2),
        # 大小 = (256, 4, 4)
        nn.Conv2d(4 * n_d_feature, 1, kernel_size=4),
        # 对数赔率张量大小 = (1, 1, 1)
        )
print(dnet)

Sequential(
  (0): ConvTranspose2d(64, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (10): Sigmoid()
)
Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(

初始化权重值

In [3]:
import torch.nn.init as init

def weights_init(m): # 用于初始化权重值的函数
    if type(m) in [nn.ConvTranspose2d, nn.Conv2d]:
        init.xavier_normal_(m.weight)
    elif type(m) == nn.BatchNorm2d:
        init.normal_(m.weight, 1.0, 0.02)
        init.constant_(m.bias, 0)

gnet.apply(weights_init)
dnet.apply(weights_init)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1))
)

In [4]:
import torch
import torch.optim

# 损失
criterion = nn.BCEWithLogitsLoss()

# 优化器
goptimizer = torch.optim.Adam(gnet.parameters(),
        lr=0.0002, betas=(0.5, 0.999))
doptimizer = torch.optim.Adam(dnet.parameters(), 
        lr=0.0002, betas=(0.5, 0.999))

# 用于测试的固定噪声,用来查看相同的潜在张量在训练过程中生成图片的变换
batch_size = 64
fixed_noises = torch.randn(batch_size, latent_size, 1, 1)

# 训练过程
epoch_num = 10
for epoch in range(epoch_num):
    for batch_idx, data in enumerate(dataloader):
        # 载入本批次数据
        real_images, _ = data
        batch_size = real_images.size(0)
        
        # 训练鉴别网络
        labels = torch.ones(batch_size) # 真实数据对应标签为1
        preds = dnet(real_images) # 对真实数据进行判别
        outputs = preds.reshape(-1)
        dloss_real = criterion(outputs, labels) # 真实数据的鉴别器损失
        dmean_real = outputs.sigmoid().mean()
                # 计算鉴别器将多少比例的真数据判定为真,仅用于输出显示
        
        noises = torch.randn(batch_size, latent_size, 1, 1) # 潜在噪声
        fake_images = gnet(noises) # 生成假数据
        labels = torch.zeros(batch_size) # 假数据对应标签为0
        fake = fake_images.detach()
                # 使得梯度的计算不回溯到生成网络,可用于加快训练速度.删去此步结果不变
        preds = dnet(fake) # 对假数据进行鉴别
        outputs = preds.view(-1)
        dloss_fake = criterion(outputs, labels) # 假数据的鉴别器损失
        dmean_fake = outputs.sigmoid().mean()
                # 计算鉴别器将多少比例的假数据判定为真,仅用于输出显示
        
        dloss = dloss_real + dloss_fake # 总的鉴别器损失
        dnet.zero_grad()
        dloss.backward()
        doptimizer.step()
        
        # 训练生成网络
        labels = torch.ones(batch_size)
                # 生成网络希望所有生成的数据都被认为是真数据
        preds = dnet(fake_images) # 把假数据通过鉴别网络
        outputs = preds.view(-1)
        gloss = criterion(outputs, labels) # 真数据看到的损失
        gmean_fake = outputs.sigmoid().mean()
                # 计算鉴别器将多少比例的假数据判定为真,仅用于输出显示
        gnet.zero_grad()
        gloss.backward()
        goptimizer.step()
        
        # 输出本步训练结果
        print('[{}/{}]'.format(epoch, epoch_num) +
                '[{}/{}]'.format(batch_idx, len(dataloader)) +
                '鉴别网络损失:{:g} 生成网络损失:{:g}'.format(dloss, gloss) +
                '真数据判真比例:{:g} 假数据判真比例:{:g}/{:g}'.format(
                dmean_real, dmean_fake, gmean_fake))
        if batch_idx % 100 == 0:
            fake = gnet(fixed_noises) # 由固定潜在张量生成假数据
            save_image(fake, # 保存假数据
                    './data/images_epoch{:02d}_batch{:03d}.png'.format(
                    epoch, batch_idx))

[0/10][0/782]鉴别网络损失:1.29204 生成网络损失:4.23079真数据判真比例:0.556864 假数据判真比例:0.471578/0.0163643
[0/10][1/782]鉴别网络损失:1.32413 生成网络损失:1.19315真数据判真比例:0.325862 假数据判真比例:0.0584459/0.326779
[0/10][2/782]鉴别网络损失:1.29984 生成网络损失:2.50886真数据判真比例:0.76801 假数据判真比例:0.604604/0.0942189
[0/10][3/782]鉴别网络损失:0.632641 生成网络损失:3.68688真数据判真比例:0.74231 假数据判真比例:0.258376/0.0282233
[0/10][4/782]鉴别网络损失:0.62855 生成网络损失:2.91273真数据判真比例:0.626551 假数据判真比例:0.0915724/0.0642669
[0/10][5/782]鉴别网络损失:0.471166 生成网络损失:2.49235真数据判真比例:0.754267 假数据判真比例:0.141809/0.0917134
[0/10][6/782]鉴别网络损失:0.44919 生成网络损失:3.6036真数据判真比例:0.846702 假数据判真比例:0.230204/0.0309037
[0/10][7/782]鉴别网络损失:0.294078 生成网络损失:3.88127真数据判真比例:0.834192 假数据判真比例:0.0927451/0.0246884
[0/10][8/782]鉴别网络损失:0.285917 生成网络损失:3.38468真数据判真比例:0.830241 假数据判真比例:0.0767035/0.0391296
[0/10][9/782]鉴别网络损失:0.220055 生成网络损失:3.81013真数据判真比例:0.910138 假数据判真比例:0.112722/0.0248739
[0/10][10/782]鉴别网络损失:0.21276 生成网络损失:3.94637真数据判真比例:0.883408 假数据判真比例:0.0775519/0.0216262
[0/10][11/782]鉴别网络损失:0.186713 生成网络损失:4.08108真数据

[0/10][93/782]鉴别网络损失:0.373368 生成网络损失:6.29217真数据判真比例:0.939781 假数据判真比例:0.248705/0.00287783
[0/10][94/782]鉴别网络损失:0.641523 生成网络损失:2.83515真数据判真比例:0.585845 假数据判真比例:0.00597657/0.0866998
[0/10][95/782]鉴别网络损失:0.289238 生成网络损失:4.64791真数据判真比例:0.971562 假数据判真比例:0.214394/0.0129243
[0/10][96/782]鉴别网络损失:0.119453 生成网络损失:5.52531真数据判真比例:0.964894 假数据判真比例:0.0707342/0.00565113
[0/10][97/782]鉴别网络损失:0.14139 生成网络损失:4.57533真数据判真比例:0.894589 假数据判真比例:0.0159728/0.0122658
[0/10][98/782]鉴别网络损失:0.104335 生成网络损失:3.81016真数据判真比例:0.949512 假数据判真比例:0.0479195/0.0274158
[0/10][99/782]鉴别网络损失:0.144172 生成网络损失:4.3311真数据判真比例:0.951773 假数据判真比例:0.0861058/0.0169303
[0/10][100/782]鉴别网络损失:0.15245 生成网络损失:4.38791真数据判真比例:0.925482 假数据判真比例:0.0676728/0.0164797
[0/10][101/782]鉴别网络损失:0.131812 生成网络损失:4.07348真数据判真比例:0.925431 假数据判真比例:0.0465029/0.0208108
[0/10][102/782]鉴别网络损失:0.13193 生成网络损失:4.23749真数据判真比例:0.945012 假数据判真比例:0.0700706/0.0167399
[0/10][103/782]鉴别网络损失:0.162345 生成网络损失:3.86853真数据判真比例:0.908036 假数据判真比例:0.0575991/0.0259303
[0/10][104/782]鉴别网络损

[0/10][186/782]鉴别网络损失:1.5708 生成网络损失:0.90502真数据判真比例:0.284187 假数据判真比例:0.0255839/0.455539
[0/10][187/782]鉴别网络损失:0.915006 生成网络损失:4.23327真数据判真比例:0.97147 假数据判真比例:0.543585/0.0201273
[0/10][188/782]鉴别网络损失:0.301587 生成网络损失:3.97319真数据判真比例:0.800424 假数据判真比例:0.0551505/0.0260464
[0/10][189/782]鉴别网络损失:0.446297 生成网络损失:2.18166真数据判真比例:0.731391 假数据判真比例:0.0817102/0.143035
[0/10][190/782]鉴别网络损失:0.523825 生成网络损失:3.03225真数据判真比例:0.866654 假数据判真比例:0.291086/0.0611929
[0/10][191/782]鉴别网络损失:0.419558 生成网络损失:3.56995真数据判真比例:0.841337 假数据判真比例:0.199078/0.0352774
[0/10][192/782]鉴别网络损失:0.289521 生成网络损失:3.47221真数据判真比例:0.862016 假数据判真比例:0.117028/0.0371247
[0/10][193/782]鉴别网络损失:0.473462 生成网络损失:2.52718真数据判真比例:0.77175 假数据判真比例:0.16282/0.0997995
[0/10][194/782]鉴别网络损失:0.341211 生成网络损失:3.25真数据判真比例:0.879659 假数据判真比例:0.178416/0.0507324
[0/10][195/782]鉴别网络损失:0.41007 生成网络损失:3.2609真数据判真比例:0.831142 假数据判真比例:0.176788/0.0456761
[0/10][196/782]鉴别网络损失:0.270738 生成网络损失:3.33079真数据判真比例:0.863413 假数据判真比例:0.103807/0.0438994
[0/10][197/782]鉴别网络损失:0.412037

[0/10][279/782]鉴别网络损失:0.43321 生成网络损失:3.1999真数据判真比例:0.816286 假数据判真比例:0.177449/0.0508964
[0/10][280/782]鉴别网络损失:0.451377 生成网络损失:2.66485真数据判真比例:0.776576 假数据判真比例:0.150559/0.0839935
[0/10][281/782]鉴别网络损失:0.56527 生成网络损失:4.18685真数据判真比例:0.82753 假数据判真比例:0.296517/0.017348
[0/10][282/782]鉴别网络损失:0.434483 生成网络损失:2.32862真数据判真比例:0.710605 假数据判真比例:0.0597562/0.112412
[0/10][283/782]鉴别网络损失:0.55455 生成网络损失:4.92043真数据判真比例:0.912405 假数据判真比例:0.35145/0.008862
[0/10][284/782]鉴别网络损失:0.453607 生成网络损失:2.84514真数据判真比例:0.696384 假数据判真比例:0.0362395/0.0685133
[0/10][285/782]鉴别网络损失:0.405559 生成网络损失:3.29952真数据判真比例:0.888617 假数据判真比例:0.234195/0.0429609
[0/10][286/782]鉴别网络损失:0.509491 生成网络损失:3.44411真数据判真比例:0.787956 假数据判真比例:0.212621/0.0383267
[0/10][287/782]鉴别网络损失:0.388517 生成网络损失:4.01663真数据判真比例:0.86429 假数据判真比例:0.202078/0.0214132
[0/10][288/782]鉴别网络损失:0.697848 生成网络损失:1.37807真数据判真比例:0.58428 假数据判真比例:0.0985626/0.296944
[0/10][289/782]鉴别网络损失:1.2465 生成网络损失:6.93095真数据判真比例:0.945977 假数据判真比例:0.660405/0.00133875
[0/10][290/782]鉴别网络损失:1.90741 生

[0/10][372/782]鉴别网络损失:0.751168 生成网络损失:3.22636真数据判真比例:0.915484 假数据判真比例:0.450743/0.0532589
[0/10][373/782]鉴别网络损失:0.536233 生成网络损失:3.84224真数据判真比例:0.814323 假数据判真比例:0.261369/0.0262867
[0/10][374/782]鉴别网络损失:0.710363 生成网络损失:2.99049真数据判真比例:0.697479 假数据判真比例:0.255994/0.0628544
[0/10][375/782]鉴别网络损失:0.847725 生成网络损失:2.46595真数据判真比例:0.647194 假数据判真比例:0.295476/0.104106
[0/10][376/782]鉴别网络损失:1.1409 生成网络损失:3.10385真数据判真比例:0.629225 假数据判真比例:0.437043/0.0591125
[0/10][377/782]鉴别网络损失:0.806329 生成网络损失:2.17749真数据判真比例:0.613626 假数据判真比例:0.202745/0.145344
[0/10][378/782]鉴别网络损失:0.701984 生成网络损失:3.83557真数据判真比例:0.833516 假数据判真比例:0.368363/0.028538
[0/10][379/782]鉴别网络损失:0.749895 生成网络损失:2.1263真数据判真比例:0.597391 假数据判真比例:0.132098/0.151948
[0/10][380/782]鉴别网络损失:0.857584 生成网络损失:4.48537真数据判真比例:0.874292 假数据判真比例:0.456779/0.0172287
[0/10][381/782]鉴别网络损失:0.930595 生成网络损失:2.00946真数据判真比例:0.500059 假数据判真比例:0.0923317/0.181349
[0/10][382/782]鉴别网络损失:0.916456 生成网络损失:4.31952真数据判真比例:0.855223 假数据判真比例:0.482803/0.0187111
[0/10][383/782]鉴别网络损失:0.6778

...

[9/10][397/782]鉴别网络损失:0.935616 生成网络损失:1.7848真数据判真比例:0.5005 假数据判真比例:0.0948863/0.214718
[9/10][398/782]鉴别网络损失:0.906968 生成网络损失:0.87734真数据判真比例:0.605788 假数据判真比例:0.278336/0.454226
[9/10][399/782]鉴别网络损失:1.21801 生成网络损失:3.24589真数据判真比例:0.86329 假数据判真比例:0.617951/0.0514158
[9/10][400/782]鉴别网络损失:1.12722 生成网络损失:1.27381真数据判真比例:0.398314 假数据判真比例:0.0925956/0.314097
[9/10][401/782]鉴别网络损失:0.637819 生成网络损失:2.04546真数据判真比例:0.863492 假数据判真比例:0.348797/0.153733
[9/10][402/782]鉴别网络损失:0.800486 生成网络损失:1.98433真数据判真比例:0.686656 假数据判真比例:0.292074/0.176923
[9/10][403/782]鉴别网络损失:0.721105 生成网络损失:1.61882真数据判真比例:0.686622 假数据判真比例:0.257382/0.232382
[9/10][404/782]鉴别网络损失:0.688229 生成网络损失:2.03854真数据判真比例:0.780419 假数据判真比例:0.31517/0.16652
[9/10][405/782]鉴别网络损失:0.730534 生成网络损失:1.96758真数据判真比例:0.702148 假数据判真比例:0.274586/0.165781
[9/10][406/782]鉴别网络损失:0.67087 生成网络损失:1.83204真数据判真比例:0.712084 假数据判真比例:0.241346/0.194486
[9/10][407/782]鉴别网络损失:0.832044 生成网络损失:2.02651真数据判真比例:0.718335 假数据判真比例:0.345512/0.163101
[9/10][408/782]鉴别网络损失:0.61646 生成网络损失:1

[9/10][491/782]鉴别网络损失:1.05553 生成网络损失:1.39155真数据判真比例:0.58017 假数据判真比例:0.33636/0.291565
[9/10][492/782]鉴别网络损失:0.808538 生成网络损失:1.86114真数据判真比例:0.716671 假数据判真比例:0.327692/0.176368
[9/10][493/782]鉴别网络损失:0.666902 生成网络损失:2.65518真数据判真比例:0.799036 假数据判真比例:0.327971/0.082643
[9/10][494/782]鉴别网络损失:1.22132 生成网络损失:0.663651真数据判真比例:0.421477 假数据判真比例:0.183691/0.542182
[9/10][495/782]鉴别网络损失:1.2541 生成网络损失:2.69736真数据判真比例:0.82758 假数据判真比例:0.575966/0.0958754
[9/10][496/782]鉴别网络损失:1.10823 生成网络损失:1.33099真数据判真比例:0.492294 假数据判真比例:0.240058/0.30136
[9/10][497/782]鉴别网络损失:0.903158 生成网络损失:3.29851真数据判真比例:0.863986 假数据判真比例:0.492186/0.0487382
[9/10][498/782]鉴别网络损失:1.43722 生成网络损失:0.825939真数据判真比例:0.334093 假数据判真比例:0.137021/0.483941
[9/10][499/782]鉴别网络损失:1.20042 生成网络损失:2.26549真数据判真比例:0.814478 假数据判真比例:0.579412/0.136906
[9/10][500/782]鉴别网络损失:0.828121 生成网络损失:2.56192真数据判真比例:0.704543 假数据判真比例:0.338487/0.100166
[9/10][501/782]鉴别网络损失:0.887384 生成网络损失:1.25331真数据判真比例:0.529149 假数据判真比例:0.138261/0.339275
[9/10][502/782]鉴别网络损失:0.932083 生成网络损失:2

[9/10][585/782]鉴别网络损失:0.846005 生成网络损失:0.967937真数据判真比例:0.582232 假数据判真比例:0.199736/0.423912
[9/10][586/782]鉴别网络损失:1.01044 生成网络损失:2.5824真数据判真比例:0.83083 假数据判真比例:0.506152/0.0987768
[9/10][587/782]鉴别网络损失:0.864257 生成网络损失:1.62425真数据判真比例:0.581629 假数据判真比例:0.212137/0.225956
[9/10][588/782]鉴别网络损失:0.917696 生成网络损失:1.52263真数据判真比例:0.662003 假数据判真比例:0.331539/0.255632
[9/10][589/782]鉴别网络损失:0.838771 生成网络损失:2.44516真数据判真比例:0.793808 假数据判真比例:0.403699/0.108766
[9/10][590/782]鉴别网络损失:0.940887 生成网络损失:1.35418真数据判真比例:0.514481 假数据判真比例:0.15488/0.31839
[9/10][591/782]鉴别网络损失:1.07797 生成网络损失:2.27433真数据判真比例:0.789769 假数据判真比例:0.495609/0.131213
[9/10][592/782]鉴别网络损失:1.01495 生成网络损失:1.37608真数据判真比例:0.542233 假数据判真比例:0.249614/0.287913
[9/10][593/782]鉴别网络损失:0.911363 生成网络损失:2.43643真数据判真比例:0.816336 假数据判真比例:0.464452/0.108567
[9/10][594/782]鉴别网络损失:0.70599 生成网络损失:1.71099真数据判真比例:0.621371 假数据判真比例:0.162442/0.226205
[9/10][595/782]鉴别网络损失:0.79167 生成网络损失:1.50119真数据判真比例:0.706821 假数据判真比例:0.307362/0.253682
[9/10][596/782]鉴别网络损失:0.728375 生成网络损失:2

[9/10][679/782]鉴别网络损失:0.724319 生成网络损失:1.26841真数据判真比例:0.694911 假数据判真比例:0.262677/0.31963
[9/10][680/782]鉴别网络损失:1.05531 生成网络损失:2.37763真数据判真比例:0.78968 假数据判真比例:0.506392/0.112496
[9/10][681/782]鉴别网络损失:0.63711 生成网络损失:2.26476真数据判真比例:0.708179 假数据判真比例:0.204319/0.127633
[9/10][682/782]鉴别网络损失:0.848134 生成网络损失:1.05319真数据判真比例:0.563437 假数据判真比例:0.173001/0.400667
[9/10][683/782]鉴别网络损失:0.883436 生成网络损失:2.5019真数据判真比例:0.851248 假数据判真比例:0.469385/0.102501
[9/10][684/782]鉴别网络损失:0.584417 生成网络损失:2.31745真数据判真比例:0.711986 假数据判真比例:0.189478/0.122776
[9/10][685/782]鉴别网络损失:0.639871 生成网络损失:1.54473真数据判真比例:0.679296 假数据判真比例:0.164755/0.248378
[9/10][686/782]鉴别网络损失:0.674671 生成网络损失:2.38554真数据判真比例:0.851897 假数据判真比例:0.381156/0.104308
[9/10][687/782]鉴别网络损失:0.621958 生成网络损失:2.29454真数据判真比例:0.739021 假数据判真比例:0.229867/0.127517
[9/10][688/782]鉴别网络损失:1.58664 生成网络损失:1.76988真数据判真比例:0.483991 假数据判真比例:0.467528/0.21385
[9/10][689/782]鉴别网络损失:1.0657 生成网络损失:1.17025真数据判真比例:0.502755 假数据判真比例:0.260034/0.352262
[9/10][690/782]鉴别网络损失:0.887557 生成网络损失:3.6

[9/10][773/782]鉴别网络损失:0.569462 生成网络损失:2.03907真数据判真比例:0.769163 假数据判真比例:0.240419/0.147365
[9/10][774/782]鉴别网络损失:0.787627 生成网络损失:1.55609真数据判真比例:0.65195 假数据判真比例:0.261537/0.241066
[9/10][775/782]鉴别网络损失:0.780326 生成网络损失:2.26068真数据判真比例:0.78142 假数据判真比例:0.36203/0.136884
[9/10][776/782]鉴别网络损失:0.62834 生成网络损失:1.78419真数据判真比例:0.682053 假数据判真比例:0.171264/0.210159
[9/10][777/782]鉴别网络损失:0.625242 生成网络损失:2.45028真数据判真比例:0.843142 假数据判真比例:0.340998/0.104004
[9/10][778/782]鉴别网络损失:0.986272 生成网络损失:1.47194真数据判真比例:0.570482 假数据判真比例:0.284118/0.26101
[9/10][779/782]鉴别网络损失:0.656693 生成网络损失:2.38108真数据判真比例:0.810578 假数据判真比例:0.329684/0.116041
[9/10][780/782]鉴别网络损失:1.05259 生成网络损失:1.29569真数据判真比例:0.527688 假数据判真比例:0.266197/0.310944
[9/10][781/782]鉴别网络损失:0.927325 生成网络损失:2.04409真数据判真比例:0.623287 假数据判真比例:0.330075/0.161399
